## Exploring Music from Spotify

This notebook includes a set of hidden "chunks" of code that, with an edit of the variables directly below, let you explore music as played through Spotify. The code in this notebook calls out to Spotify which returns the requested data. Descriptions of the technical specs can be found at [Spotify Web API](https://developer.spotify.com/documentation/web-api) and the description of the Python package we use to access the data can be found at [Spotipy](https://spotipy.readthedocs.io/en/2.22.1/). 

In [1]:
user_artist = 'one direction'

In [4]:
# Packages
import requests # send API
import base64   # format text
import json     # translate JOSN formatted data received from Spotify to pandas
import pandas as pd

import plotly.graph_objects as go

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

# Spotify API credentials - normally I would not have these in the code!
CLIENT_ID ='cac7108fe0674270a2e06785ff229638'
CLIENT_SECRET ='ebe70aaa3ae9474fbd22fa41478b2761'

<h1 style="font-size:14px; background-color:hsla(206, 53%, 32%, 0.8); color: white;">Cell 2: This block of code declares functions and sets credentials.</h1>

In [5]:
# Function to assemble query
#def construct_url(url, query_type, ):
#    return url

# Function to obtain access token
def get_access_token(CLIENT_ID, CLIENT_SECRET):
    # Spotify URL for authentication
    auth_url = 'https://accounts.spotify.com/api/token'
    client_creds = f'{CLIENT_ID}:{CLIENT_SECRET}'
    client_creds_b64 = base64.b64encode(client_creds.encode())
    token_headers = {'Authorization': f'Basic {client_creds_b64.decode()}'}
    token_data = {'grant_type': 'client_credentials'}
    r = requests.post(auth_url, headers=token_headers, data=token_data)
    token_response_data = r.json()
    access_token = token_response_data.get('access_token')
    return access_token

# Function to call Spotify data
def query_Spotify(access_token, url):
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)
    return response.json()

# Permissions
access_token = get_access_token(CLIENT_ID, CLIENT_SECRET) # for direct CURL access rather than via Spotipy

# Scope
# https://developer.spotify.com/documentation/web-api/concepts/scopes
# https://pypi.org/project/spotipy/
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,
                                                           client_secret=CLIENT_SECRET)) # vs. SpotifyOAuth()

#client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
#sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


<h1 style="font-size:14px; background-color:hsla(206, 53%, 32%, 0.8); color: white;">Cell 3: This block of code retrieves artist identifications and prepares searches based on the entry under the input field `user_artist`.</h1>

In [6]:
result = sp.search(user_artist) 
artist_full_uri = result['tracks']['items'][0]['artists'][0]['uri']
artist_uri = artist_full_uri.split(":")[2]

top_track_url = f'https://api.spotify.com/v1/artists/{artist_uri}/top-tracks?market=US'


<h1 style="font-size:14px; background-color:hsla(1, 53%, 32%, 0.8); color: white;">Cell 4: This block of code produces a list of recent albums for the artist lised in `user_artist`.</h1>

In [7]:
result = sp.search(user_artist) 
artist_uri = result['tracks']['items'][0]['artists'][0]['uri']

sp_albums = sp.artist_albums(artist_uri, album_type='album', country='US')
name = []
uri = []
for i in sp_albums['items']:
    name.append(i['name'])
    uri.append(i['uri'])

#print(json.dumps(sp_albums['items'][0]['name'], indent=4))
album_names_df = pd.DataFrame(name)
album_names_df.columns =['AlbumName']
album_names_df["URI"] = uri
print(album_names_df['AlbumName'])


0    Made In The A.M. (Deluxe Edition)
1                        FOUR (Deluxe)
2           Midnight Memories (Deluxe)
3      Take Me Home (Expanded Edition)
4                         Up All Night
Name: AlbumName, dtype: object


<h1 style="font-size:14px; background-color:hsla(1, 53%, 32%, 0.8); color: white;">Cell 5: This block of code produces a list of most popular songs for the artist lised in `user_artist`.</h1>

In [8]:
# Query Spotify without Spotipy package
top_tracks_json = query_Spotify(access_token, top_track_url)
#print(json.dumps(top_tracks_json, indent=4))

spotify_results_df = pd.DataFrame(top_tracks_json['tracks'])
spotify_results_df.head(100)

# https://plotly.com/python/graph-objects/

layout = dict(plot_bgcolor='white',
#              margin=dict(t=20, l=20, r=20, b=20),
              xaxis=dict(title='name', linecolor='#d9d9d9'),
              yaxis=dict(title='popularity',linecolor='#d9d9d9', range=[0, 100])) 

data = go.Bar(x=spotify_results_df['name'],
                  y=spotify_results_df['popularity']
                  )

plot_title = f"Popular {user_artist} Songs<br><sup>Top Ten</sup>"
fig = go.Figure(data=data, layout=layout)
fig.update_layout(title=go.layout.Title(text=plot_title))
fig.show()

<h1 style="font-size:14px; background-color:hsla(1, 53%, 32%, 0.8); color: white;">Cell 6: This block of code produces a list of most popular songs for the artist lised in `user_artist` by song length.</h1>

In [9]:
# Query Spotify using Spotipy package
#results_json = sp.search(q="artist:tears for fears",  type='track',  limit=10)
results_json = sp.search(q=f"artist:{user_artist}",  type='track',  limit=10)

results_df = pd.DataFrame(results_json['tracks']['items'])
tracks_df = results_df[['name', 'popularity', 'duration_ms']].copy()
tracks_df['duration'] = round(tracks_df['duration_ms']/1000)

layout = dict(plot_bgcolor='white',
#              margin=dict(t=20, l=20, r=20, b=20),
              xaxis=dict(title='Duration', linecolor='#d9d9d9'),
              yaxis=dict(title='Popularity',linecolor='#d9d9d9')) 

data = go.Scatter(x=tracks_df['duration'],
                  y=tracks_df['popularity'],
                  text=tracks_df['name'],
                  textposition='top left',
                  textfont=dict(color='black'),
                  mode='markers+text')

plot_title = f"Popular {user_artist} Songs by Duration<br><sup>(in seconds)</sup>"
fig = go.Figure(data=data, layout=layout)
fig.update_layout(title=go.layout.Title(text=plot_title))

fig.show()

<h1 style="font-size:14px; background-color:hsla(1, 53%, 32%, 0.8); color: white;">Cell 7: This block of code produces a list of songs from the selected albums for the selected artist and retrieves song data.</h1>

In [10]:
album_selection = ['Made In The A.M. (Deluxe Edition)','Midnight Memories (Deluxe)']

In [11]:
# remove unmatching albums from album_names_df
filtered_album_names_df = album_names_df[album_names_df.AlbumName.isin(album_selection) == True].copy()
# remove uri's from uri that do not exist in filtered data frame
filtered_uri = [item for item in uri if item in filtered_album_names_df['URI'].values]
#print(json.dumps(filtered_uri, indent=4))

# Loop to get song URI for selected albums
song_name = []
song_uri = []
album = []
count = 0
for j in filtered_uri:    
    tracks = sp.album_tracks(j, market='US')   
    for i in tracks['items']:
        album.append(name[count])
        song_name.append(i['name'])
        song_uri.append(i['uri'])
    count+=1
song_name

acoustic = []; dance = []; energy = []; instrumental = []; liveness = []; loudness = []; speech = []; tempo = []; valence = []

# Loop throug to get song qualities
for i in song_uri:
    feat = sp.audio_features(i)[0]
    acoustic.append(feat['acousticness'])
    dance.append(feat['danceability'])
    energy.append(feat['energy'])
    speech.append(feat['speechiness'])
    instrumental.append(feat['instrumentalness'])
    loudness.append(feat['loudness'])
    tempo.append(feat['tempo'])
    liveness.append(feat['liveness'])
    valence.append(feat['valence'])

#print(json.dumps(song_name, indent=4))
songs_df = pd.DataFrame({'Name': song_name, 'URI': song_uri, 'acoustic': acoustic,'danceability': dance,
                        'energy': energy,'speechiness': speech,'instrumentalness': instrumental,
                        'loudness': loudness, 'tempo': tempo, 'liveness': liveness,'valence': valence})

songs_df.head(10)

,Name,URI,acoustic,danceability,energy,speechiness,instrumentalness,loudness,tempo,liveness,valence
0,Hey Angel,spotify:track:4fLTtg48X0QtKrj0Jv8I3v,0.020,0.523,0.883,0.036,0.000e+00,-2.956,92.215,0.116,0.350
1,Drag Me Down,spotify:track:2K87XMYnUMqLcX3zvtAF4G,0.109,0.730,0.703,0.037,0.000e+00,-5.672,138.113,0.066,0.595
2,Perfect,spotify:track:3NLnwwAQbbFKcEcV8hDItk,0.060,0.647,0.823,0.076,0.000e+00,-5.231,99.933,0.119,0.396
3,Infinity,spotify:track:6N5xh0tYYLTQRiCCaNbAUt,0.069,0.537,0.705,0.034,0.000e+00,-6.073,131.965,0.084,0.320
4,End of the Day,spotify:track:5AezhHjX3R1bmxcAEgrFpS,0.026,0.502,0.594,0.036,0.000e+00,-5.031,153.687,0.083,0.349
5,If I Could Fly,spotify:track:5CRVwXGikmqzoRO6q7FeAg,0.781,0.565,0.245,0.026,0.000e+00,-9.132,75.055,0.100,0.307
6,Long Way Down,spotify:track:2UFOh8iMQ2DON7sl3lqGJH,0.077,0.633,0.565,0.024,2.310e-05,-5.179,145.001,0.168,0.539
7,Never Enough,spotify:track:7rdP71W8x4UbJ0KAKBACbo,0.038,0.579,0.779,0.051,0.000e+00,-1.829,164.145,0.385,0.587
8,Olivia,spotify:track:5dONhl6aXFuN86UBdayF6W,0.044,0.640,0.681,0.033,0.000e+00,-5.362,118.062,0.166,0.676
9,What a Feeling,spotify:track:0iSWAT0EL8TwmzcgBjKMh6,0.149,0.678,0.869,0.046,0.000e+00,-4.345,123.964,0.059,0.882


<h1 style="font-size:14px; background-color:hsla(1, 53%, 32%, 0.8); color: white;">Cell 8: This block of code creates a chart of the song properties.</h1>

In [12]:
layout = dict(plot_bgcolor='white',
#              margin=dict(t=20, l=20, r=20, b=20),
              xaxis=dict(title='Danceability', linecolor='#d9d9d9'),
              yaxis=dict(title='Energy',linecolor='#d9d9d9')) 


data = go.Scatter(x=songs_df['danceability'],
                  y=songs_df['energy'],
                  marker=dict(size=songs_df['tempo'], 
#                  color=np.random.randn(500), #set color equal to a variable
                  colorscale='Viridis', # one of plotly colorscales
                  showscale=True,
                  color=songs_df['loudness']),
                  text=songs_df['Name'],
                  textfont=dict(color='black'),
                  mode='markers')

plot_title = f"Selected {user_artist} Song Evaluation<br><sup>Size is tempo</sup>"
fig = go.Figure(data=data, layout=layout)
fig.update_layout(title=go.layout.Title(text=plot_title))

fig.show()

<h1 style="font-size:14px; background-color:hsla(1, 53%, 32%, 0.8); color: white;">Cell 9: This block of code creates a list of song recommendation based upon the tracks from the selected albums.</h1>

In [13]:
#https://developer.spotify.com/documentation/web-api/reference/get-recommendations
song_uri_only = [entry.split(':')[2] for entry in song_uri]
first_three = [song_uri_only[0], song_uri_only[1], song_uri_only[2]]

results_json = sp.recommendations(seed_tracks=first_three, market='US', limit=10)

for track in results_json['tracks']:
    print(track['name'], "by", track['artists'][0]['name'])


5:15 by Bridgit Mendler
Sunshine - From the Motion Picture “Ron’s Gone Wrong” by Liam Payne
No Control by One Direction
Walk Away by High School Musical Cast
Wonder by Shawn Mendes
The Middle by Zedd
I Think I'm In Love by Kat Dahlia
Stone Cold by Demi Lovato
Superhero by Austin Moon
If the World Was Ending (feat. Julia Michaels) by JP Saxe


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=03da49fa-82ee-445f-b697-49f5814fb990' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>